In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-titles-csv/movie_titles.csv
/kaggle/input/pivot-table-csv/pivot_table.csv
/kaggle/input/naucno-izracunavanje/pivot_table.csv
/kaggle/input/naucno-izracunavanje/test_data.csv


In [2]:
from surprise import Dataset, Reader, KNNBasic, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
import pickle

In [3]:
import pandas as pd
import numpy as np
import time
df_pivot = pd.read_csv('/kaggle/input/naucno-izracunavanje/pivot_table.csv', index_col=0)
print(df_pivot.columns)

Index(['3', '8', '16', '17', '18', '26', '28', '30', '32', '33',
       ...
       '4472', '4474', '4478', '4479', '4485', '4488', '4490', '4492', '4493',
       '4496'],
      dtype='object', length=1350)


In [4]:
df_melt = df_pivot.stack().reset_index().rename(columns={'level_1': 'Movie_Id', 0: 'Rating'})
print(df_melt.columns)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_melt[['Cust_Id', 'Movie_Id', 'Rating']], reader)

Index(['Cust_Id', 'Movie_Id', 'Rating'], dtype='object')


In [5]:
trainset, validset = train_test_split(data, test_size=0.25)

In [6]:
similarity_options = {'name': 'cosine', 'user_based': False}
best_rmse = float('inf')
best_model = None

In [7]:
for k in [25, 30, 35, 40, 45]:
    algo = KNNBasic(k=k, sim_options=similarity_options)
    start_time = time.time()
    algo.fit(trainset)
    end_time = time.time()
    print(f"Vreme fitovanja: {end_time - start_time:.2f} sekundi")
    start_time = time.time()
    predictions = algo.test(validset)
    end_time = time.time()
    print(f"Vreme predvidjanja: {end_time - start_time:.2f} sekundi")
    rmse = accuracy.rmse(predictions, verbose=False)
    print(f'Cosine similarity, k={k}, RMSE={rmse}')
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = algo
        best_params = {'similarity': 'cosine', 'k': k}


Computing the cosine similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 48.74 sekundi
Vreme predvidjanja: 602.77 sekundi
Cosine similarity, k=25, RMSE=0.9660813877265495
Computing the cosine similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 44.65 sekundi
Vreme predvidjanja: 645.77 sekundi
Cosine similarity, k=35, RMSE=0.9663931855621274
Computing the cosine similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 46.05 sekundi
Vreme predvidjanja: 655.85 sekundi
Cosine similarity, k=40, RMSE=0.9672557794827426
Computing the cosine similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 47.34 sekundi
Vreme predvidjanja: 675.85 sekundi
Cosine similarity, k=45, RMSE=0.9682685972232347


In [8]:
similarity_options['name'] = 'pearson'
for k in [10, 15, 18, 20, 25]:
    algo = KNNBasic(k=k, sim_options=similarity_options)
    start_time = time.time()
    algo.fit(trainset)
    end_time = time.time()
    print(f"Vreme fitovanja: {end_time - start_time:.2f} sekundi")
    start_time = time.time()
    predictions = algo.test(validset)
    end_time = time.time()
    print(f"Vreme predvidjanja: {end_time - start_time:.2f} sekundi")
    rmse = accuracy.rmse(predictions, verbose=False)
    print(f'Pearson similarity, k={k}, RMSE={rmse}')
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = algo
        best_params = {'similarity': 'pearson', 'k': k}

Computing the pearson similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 57.99 sekundi
Vreme predvidjanja: 471.21 sekundi
Pearson similarity, k=10, RMSE=0.9319699755307492
Computing the pearson similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 53.95 sekundi
Vreme predvidjanja: 517.78 sekundi
Pearson similarity, k=15, RMSE=0.9262130085222333
Computing the pearson similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 58.67 sekundi
Vreme predvidjanja: 540.83 sekundi
Pearson similarity, k=18, RMSE=0.9257038903973615
Computing the pearson similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 58.67 sekundi
Vreme predvidjanja: 562.84 sekundi
Pearson similarity, k=20, RMSE=0.9259095324484256
Computing the pearson similarity matrix...
Done computing similarity matrix.
Vreme fitovanja: 57.84 sekundi
Vreme predvidjanja: 592.24 sekundi
Pearson similarity, k=25, RMSE=0.9273933957160797


In [9]:
with open('best_knn_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print(f'Najbolji model koristi {best_params["similarity"]} sličnost sa k={best_params["k"]} i RMSE={best_rmse}')


Najbolji model koristi pearson sličnost sa k=18 i RMSE=0.9257038903973615


In [10]:
test_data = pd.read_csv('/kaggle/input/naucno-izracunavanje/test_data.csv')
test_data = Dataset.load_from_df(test_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)
testset = test_data.build_full_trainset().build_testset()

with open('best_knn_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

predictions = best_model.test(testset)
final_rmse = accuracy.rmse(predictions)

RMSE: 1.0623
